# Overall Analysis Plan
* **A. Importing & cleaning the data**
* **B. Manipulating the data: 3 questions**
    * What are across countries products with the highest level of energy? and what are the top 10 for the main countries? What are the Top 20 Energy products within the 75% most common energy levels?
    * What is the proportion of products with additives? Is there a pattern between high calory products and number of additives
    * What are the products with the most balanced levels of core nutrients? What is their level of calory? What is their breakdown of core nutrients?
* **C. Text data**
* **D. Time series analysis**
* **E. Buidling a database**
* **F. Finding correlations**

## A. Importing & Cleaning the Data

### Importing libraries and data

In [1]:
# Importing the file and creating a dataframe
OFmaster=pd.read_csv("C:/Users/fbaff/Documents/Data Science/EPFL/0_Datasets/foodfacts/en.openfoodfacts.org.products.tsv",low_memory=False ,sep='\t')

NameError: name 'pd' is not defined

In [ ]:
OFmaster.head()

### Cleaning data plan
* Address NaN
* Dealing with duplicates or incorrect values
* Adapting data format
* Detecting and addressing outliers
* Reference of header descriptions at:https://world.openfoodfacts.org/data/data-fields.txt

In [ ]:
# Understanding the data structure
OFmaster.shape

In [ ]:
# Listing the different types of data
OFmaster.dtypes

In [ ]:
# Evaluating number of records per column
OFmaster.notnull().sum()

#### Adress NaN issues (principles)
* Drop columns for which there is no data at all
* Drop products (i.e rows) for which there are too many missing data for the variables supporting their unique identification;
* i.e variables in the "General information" section: https://world.openfoodfacts.org/data/data-fields.txt
* It will help identifying and addressing duplicates afterwards

In [ ]:
# remove variables which do not have value at all (i.e Not null = 0)
df1=OFmaster.dropna(how='all', axis=1)
df1.shape # 356027 rows (no change); 16 columns have been removed (147 vs 163)

In [ ]:
# Check remaining NaN values per column
df1.isnull().sum()

In [ ]:
# drop NaN for variable which are critical for identifying a product
df2=df1.dropna(subset=["code","creator","created_datetime","product_name","ingredients_text"])
df2.isnull().sum()

In [ ]:
# Check NaN for variables that we will integrate in the question "Text Data"
df2["ingredients_text"].isnull().sum()

In [ ]:
# It remains 282069 records (out of 356027) which is an acceptable trade-off regarding the quality of the dataset
# We may decide to include back the records with missing value related to ingredient text for increasing the size of the reference data
df2.shape

In [ ]:
df2.describe()

#### Analysis of duplicates on a selection of variables
* bar code

In [2]:
# Check if there are duplicate bar codes (i.e variable "code")
codedup = df2["code"]
df2[codedup.isin(codedup[codedup.duplicated()])]
# There is no duplicate in the column "code"

NameError: name 'df2' is not defined

#### Adapting data format
* Transform dates variable related object into proper date format
* Aligning the name of countries

In [3]:
# Change the following variables: created_datetime, last_modified_datetime
df2['created_datetime'] = pd.to_datetime(df2['created_datetime'])
df2['last_modified_datetime'] = pd.to_datetime(df2['last_modified_datetime'])

NameError: name 'pd' is not defined

In [ ]:
df2

In [4]:
# Explore the list of countries
df2['countries'].value_counts()

NameError: name 'df2' is not defined

In [ ]:
# define a function to align the name of the following countries:
def newname(s):
    s=s.str.replace('US','United States')
    s=s.str.replace('Suisse','Switzerland')
    s=s.str.replace('Deutschland','Germany')
    s=s.str.replace('España','Spain')
    s=s.str.replace('en:FR,France','France')
    s=s.str.replace('Россия','Russia')
    s=s.str.replace('en:CH','Switzerland')
    s=s.str.replace('en:US','United States')
    s=s.str.replace('Schweiz','Switzerland')
    s=s.str.replace('en:FR','France')
    s=s.str.replace('en:United States','United States')
    s=s.str.replace('en:GB','United Kingdom')
    return s
# there are country names which would need to be changed. The current command address the vast majority of observations which
# will be covered by the analysis afterwards on the Top countries (Manipulating data, question 1)

In [ ]:
# Create a new dataframe for the modifying the name of countries
new_countries=df2['countries']

In [ ]:
# apply the "newname" function to new countries
new_countries=newname(new_countries)
new_countries.value_counts()

In [ ]:
# Integrate the "new-countries" as a new column in df2
df2['new_countries']=new_countries

In [ ]:
# drop the column 'countries'
df2.drop(['countries'], axis=1)

#### Detecting and addressing outliers for the following variables:
* Overall assumptions: if outliers are explained by some incorrect reported figures we will remove the product from the analysis
* We will take 2 variables depending on a multiple factors to evaluate the integrity of the database:
* energy_100g (267,005 obervations) - Conclusion:  35 identified outliers (Sigma>4). After a manual check of about 10 data points (in and out outliers), it seems that reported figures seemed to be correct. We keep these products.
* cholesterol_100g (143,614 observations) - Conclusion 8 identified outliers with Sigma>3. We keep as it is very small number
* Overall conclusion: we keep all products at this stage based on the analysis of outliers

In [5]:
df2.describe()

NameError: name 'df2' is not defined

##### Assess outliers for the variable energy_100g

In [6]:
# let's try to define the function which will help to identfy data with a std deviation >4
# energy_100g
filter0 = np.abs(df2['energy_100g'] - df2['energy_100g'].mean()) > (4 * df2['energy_100g'].std())
filter0.sum() # 35 products

NameError: name 'np' is not defined

In [ ]:
# check the list of outliers with an energy value >4 Sigma
outliers=df2[filter0]
outliers_check=outliers[['code', 'product_name', 'energy_100g']]
outliers_check.sort_values(['energy_100g'], ascending=False).head()
# after having checked about 10 products (reported figures vs other databases), it appears that the majority is correct.
# We will keep all products (even if some mistakes in reporting the figures are likely to have happened for some inputs)

##### Assess outliers for the variable: cholesterol_100g

In [7]:
df2['cholesterol_100g'].count()

NameError: name 'df2' is not defined

In [ ]:
# let's try to define the function which will help to identfy data with a std deviation >3
# energy_100g
filter1 = np.abs(df2['cholesterol_100g'] - df2['cholesterol_100g'].mean()) > (3 * df2['cholesterol_100g'].std())
filter1.sum() # 8 products

## B. Manipulating the data
* What are across countries products with the highest level of energy? and what are the top 10 for the main countries? What are the Top 20 Energy products within the 75% most common energy levels?
* What is the proportion of products with additives? Is there a pattern between high calory products and number of additives?
* What are the products with the most balanced levels of core nutrients? What is their level of calory? What is their breakdown of core nutrients?

### What are across countries products with the highest level of energy? and what are the top 10 for the main countries?

In [ ]:
# create a summarized dataframe
nrj=df2[['code','product_name','new_countries','energy_100g','proteins_100g','carbohydrates_100g','sugars_100g','fat_100g',"sodium_100g"]]
nrj.head()

In [8]:
nrj.isnull().sum()

NameError: name 'nrj' is not defined

In [ ]:
# drop NaN for energy_100g
nrj_clean=nrj.dropna(subset=['energy_100g'])
nrj_clean.isnull().sum()

In [ ]:
nrj_clean.describe()

In [9]:
# sort energy_100g values (descending)
nrj.sort_values(['energy_100g'], ascending=False)

NameError: name 'nrj' is not defined

In [10]:
# check the countries with the highest number of observations: Top 3 US, France, Switzerland
nrj['new_countries'].value_counts()

NameError: name 'nrj' is not defined

In [ ]:
# Top 10 highest energy products listed in the US only
top10US=nrj.loc[nrj.new_countries=='United States']
top10US[['product_name','energy_100g','new_countries']].head()

In [ ]:
# change the index to get the country names
top10US.set_index('product_name', inplace=True)

In [ ]:
# select the data to plot
v1US = top10US.sort_values(['energy_100g'], ascending=False)[0:10]['energy_100g']
v1US

In [ ]:
# Plot the Top 10 products for the US
bar_plotUS = v1US.plot.barh()
bar_plotUS.set_title('US - Top 10 products with highest level of energy_100g (kj)', fontweight="bold")
bar_plotUS.set_xlabel('Level of calories in Kj') # x label
bar_plotUS.invert_yaxis() # invert the y axis to get the highest difficulty first

#### Key takeaways
* Most energetic product reported in the US are transformed products with a lot of sugars/fat and/or concentrated products

In [ ]:
# Top 10 highest energy products listed in the France only
top10FR=nrj.loc[nrj.new_countries=='France']
top10FR[['product_name','energy_100g','new_countries']].head()

In [ ]:
# change the index to get the country names
top10FR.set_index('product_name', inplace=True)

In [ ]:
# select data for the plot
v1FR=top10FR.sort_values(['energy_100g'], ascending=False)[0:10]['energy_100g']
v1FR

In [ ]:
# Plot the Top 10 products for France
bar_plotFR = v1FR.plot.barh()
bar_plotFR.set_title('FRANCE - Top 10 products with highest level of energy_100g (kj)', fontweight="bold")
bar_plotFR.set_xlabel('Level of calories in Kj') # x label
bar_plotFR.invert_yaxis() # invert the y axis to get the highest difficulty first

#### Key takeaways
* The overall level of calories (Kj) for top energetic products reported in France are much lower that the most energetic products in the US
* Top 3 level of calories is driven by fat, proteins and/or sugars

In [ ]:
# Top 10 highest energy products listed in the Switzerland only
top10CH=nrj.loc[nrj.new_countries=='Switzerland']
top10CH[['product_name','energy_100g','new_countries']].head()

In [11]:
# change the index to get the product names
top10CH.set_index('product_name', inplace=True)

NameError: name 'top10CH' is not defined

In [ ]:
# select data for the plot
v1CH=top10CH.sort_values(['energy_100g'], ascending=False)[0:10]['energy_100g']
v1CH
# not relevant to plot CH data as they have all the same value

#### Key takeaways
* Majority of high level calory for prodducts reported in CH are heavy fat product being part almots of the same category
* Interpretation is very limited in this case

### What are the Top 20 Energy products within the 75% most common energy levels?

In [ ]:
# Define the function which will help to identify data with a std deviation >3 (i.e outliers)
# energy_100g
filter0 = np.abs(nrj_clean['energy_100g'] - nrj_clean['energy_100g'].mean()) > (3 * nrj_clean['energy_100g'].std())
filter0.sum() # 98 products

In [ ]:
# Let's select all the rows now considered as outliers
outliers = nrj_clean.loc[filter0, :]
outliers.shape

In [ ]:
# define a new dataframe without outliers
nrj_new = nrj_clean.drop(outliers.index, axis=0)

In [ ]:
nrj_new.describe()

In [ ]:
# Let's have a look at the distribution plot of the energy variable
sns.distplot(nrj_new.energy_100g)
plt.rcParams["figure.figsize"]=[20,10]

In [ ]:
# change the index to get the product names
nrj_new.set_index('product_name', inplace=True)

In [ ]:
# Select energy data with a value < 1674 (75% of all observations)
nrj_new2=nrj_new[nrj_new['energy_100g'] < 1674]

In [ ]:
# Top 20 products with a calory level below 1674 Kj
nrj3= nrj_new2.sort_values(['energy_100g'], ascending=False)[0:20]['energy_100g']
nrj3

### Key takeaways
* Top 20 products with a calory level (Kj) with the 75% most common energy levels
######  1. Moelleux fourrés fraise
######  2. 10 Pains au Chocolat
######  3. Cheddar au poivre
######  4. Smilly Fourrage à la Fraise   
######  5. Crème de nougat blanc 
######  6. Moelleux fourré Fraise   
######  7. Mélange de Biscuits - Recette Japonaise  
######  8. Bonbons tendres aux goûts fruités   
######  9. Schoko 30% weniger Zucker              
######  10. Flora Original                           
######  11. Ficelles de Pain Recette Sésame pavot   
######  12. Flora original                            
######  13. Barres riz et blé complet chocolat        
######  14. Multi Frutti                              
######  15. Snack poppé saveur paprika                 
######  16. Schoko 30% weniger Zucker                 
######  17. Pâté de foie pur porc                     
######  18. Rosette                                   
######  19. Pesto Verde                               
######  20. Délice d'Amandes

### What is the proportion of products with additives? Is there a pattern between high calory products and number of aditives?

In [ ]:
# create a summarized dataframe
add_analysis=df2[['code','product_name','new_countries','energy_100g','additives_n','proteins_100g','carbohydrates_100g','sugars_100g','fat_100g',"sodium_100g"]]
add_analysis.shape # 282,069 products

In [ ]:
# drop NaN for energy_100g
add_analysis_clean=add_analysis.dropna(subset=['energy_100g'])
add_analysis_clean.isnull().sum()

In [ ]:
# Select energy data with values > 0
add_analysis_clean=add_analysis_clean[add_analysis_clean['energy_100g'] > 0]

In [ ]:
# Percentage of products with additives
a=add_analysis_clean['code'].count()
b=add_analysis_clean[add_analysis_clean['additives_n']>=1].code.count()
b/a*100 # 61.98% of product listed in the cleaned database include additives

In [ ]:
# Let's have a look at the distribution plot of the nb of additives
dist=add_analysis_clean[add_analysis_clean['additives_n']>=1]
sns.distplot(dist.additives_n)
plt.title('Graph 1: Distribution Plot - Number of Additives', fontweight="bold")
plt.rcParams["figure.figsize"]=[20,10]
# large majority of number of additives per product is <5

In [ ]:
add_analysis_clean['energy_100g'].shape

In [ ]:
add_analysis_clean['additives_n'].shape

In [ ]:
# Plot level of energy and number of additives (exploratory for energy values <10,000 Kj)
sns.pairplot(add_analysis_clean,  x_vars = ['energy_100g'], y_vars=['additives_n'], size = 5)
plt.title('Graph 2 - Level of Calories (Kj) vs Number of Additives',fontweight="bold")
plt.xlim([0, 10000])
plt.rcParams["figure.figsize"]=[20,10]
plt.show()

### Key takeways
* A large majority of number of additives per product is <5 (Graph 1) 
* Most of the products which have a calory level < 4000 Kj would have at least one additive (Graph 2)
* It appears that there is not necessarily a correlation between the number of additives and a high level of calories or eventually the number of additives may have an impact of a low level of calories (to be further explored in the last part of the project); Graph 2

### What are the products with the most balanced levels of core nutrients? What is their level of calory? What is their breakdown of core nutrients?

#### Preliminary remarks: What is a balanced level of nutrients?
* According to common practice, the levell of carbohydrates should be ranged between 45% and 65% of total calories 
(https://www.livescience.com/51976-carbohydrates.html)
* Additionally, WHO recommendations for adult are: less than 10% of free sugars and less than 30% of fat   
(http://www.who.int/en/news-room/fact-sheets/detail/healthy-diet)
* Even if those % are generally appplied to an average daily regime, we will try to identify products in the factfood database which would match these criteria

In [ ]:
# Let's create a new dataframe food3 based on the dataframe from the previous question
food3 =add_analysis_clean
food3.shape # 257,802 products

In [ ]:
# Create columns to compute the % of proteins, carbohydrates, fat for each given products and other calculus for visualization
food3['carbohydrates_perc']=(food3['carbohydrates_100g']/food3['energy_100g'])*100
food3['fat_perc']=(food3['fat_100g']/food3['energy_100g'])*100
food3['sugars_perc']=(food3['sugars_100g']/food3['energy_100g'])*100
food3['core_nutrients_perc']=food3['sugars_perc']+food3['carbohydrates_perc']+food3['fat_perc']
food3['Others_perc']=100-food3['core_nutrients_perc'] # variable useful for stacked bar chart
food3['Total_perc']=food3['core_nutrients_perc']+food3['Others_perc']
food3

In [ ]:
# there are some inconsistencies and/or mistakes in the way figures are reported. As a consequence, we will consider products
# with a Total_% of 100% and less for this specific question
food4=food3[food3['core_nutrients_perc'] <= 100]
food4.shape # 245,052 products

In [ ]:
# Let's define the selection rules as: Carbohydrates (45% to 65%) and Fat (20%-35%) and Protein (10%-35%)
target=food4[(food4['carbohydrates_perc'] >= 45) & (food4['carbohydrates_perc'] <= 65) & (food4['sugars_perc'] <=10) & (food4['fat_perc'] <=30)]
target.shape # 29 products are matching these criteria

In [ ]:
# change the index to get the product names
target.set_index('product_name', inplace=True)

In [ ]:
# select data for the plot (ranking based on the level of energy)
targ1=target.sort_values(['energy_100g'], ascending=False)
targ1.head(10)

In [ ]:
# Plot 29 products matching balance nutrients criteria (ranking based on their level of calories)
bar_plot1 = targ1['energy_100g'].plot.barh()
bar_plot1.set_title('Graph 3 - 29 most nutrient balanced products - Ranking based on their level of calories (kj)',fontweight="bold")
bar_plot1.set_xlabel('Level of calories in Kj') # x label
plt.rcParams["figure.figsize"]=[30,10]
bar_plot1.invert_yaxis() # invert the y axis to get the highest difficulty first
plt.show()

In [ ]:
# data
others = targ1[0:10]['Others_perc']
carbohydrates = targ1[0:10]['carbohydrates_perc']
sugars = targ1[0:10]['sugars_perc']
fat = targ1[0:10]['fat_perc']
y=np.arange(len(others))

# plot
plt.barh(y, others,color='red')
plt.barh(y,carbohydrates,color='gray', left=others)
plt.barh(y,sugars, color='green',left=list(map(lambda g, y: g+y, others,carbohydrates)))
plt.barh(y,fat, color='blue',left=list(map(lambda g, y: g+y, carbohydrates,sugars)))

# labels
y_labels=['Stevia','Farine bio de millet','Butter & Herb Mashed Potatoes','le pennette rigate N87','Enriched Macaroni Product, Fettucine','Premium jasmine Rice','Organic Sprouted Whole Wheat Flour','Thé glacé earl grey','Barbar au Miel','Chicorée']
plt.yticks(y, y_labels)

# legend
plt.legend(['others','carbohydrates', 'sugars','fat'], loc='upper right')

# set the title
plt.title('Graph 4 - Top 10 most energetic products - Breakdown of core nutrients (% of total calories/100g)', fontweight="bold")

# invert y axis
plt.gca().invert_yaxis() # invert the y axis to get the highest difficulty first
plt.rcParams["figure.figsize"]=[20,10]
plt.show()

### Key takeways
* There are 29 products in the foodfact database matching balanced nutrition criteria (Graph 3). Their energy level is below 200Kj per 100g
* Most of them could be grouped in 3 categories (Graph 3): Natural product (e.g. Stevia), limited transformed products (flour, rice, pasta) or antioxidant beverages
* If we focus on the Top 10 in terms of claories (among those 29 products), most of them are made of a majority of carbohydrates, a minority of fat and sugars, the difference coming from other constituents (Graph 4)

## C. Text Data

In [ ]:
# Let's have a look at "ingredients_lits" and transform data as strings
df2['ingredients_text'].astype(str)

In [ ]:
# define a dataframe to perform manipulation
new_ingredients = df2["ingredients_text"]

In [ ]:
# define a function to remove all unwanted elements; i.e text between brackets and then replacing the "space" with a ", "
def transform(s):
    s=s.str.replace(r'\(.*?\)', '') # text between brackets
    s=s.str.replace('  ',', ') # 2 spaces replaced by a ", "
    s=s.str.replace('\d*','') # replace '*' by a space
    s=s.str.replace('%','')# replace % by a space
    s=s.str.replace('*','') # replace * by a space
    s=s.str.replace(' -',',')
    s=s.str.replace('.','')
    s=s.str.replace(' and',',')
    return s

In [ ]:
# apply the cleaning "transform" function to new_ingredients data
new_ingredients=transform(new_ingredients)
new_ingredients.head()

In [ ]:
# create a new column "new_ingredients"
df2['new_ingredients']=new_ingredients

In [ ]:
# drop the former column "ingredients_text"
df2.drop(['ingredients_text'], axis=1)

In [ ]:
# split the list of ingredients into individual columns (+ make sure we have strings)
df3=(df2['new_ingredients'].str.split(',', expand=True).rename(columns=lambda x: f"ingredients_{x+1}"))
df3.astype(str)

In [ ]:
# select most common group of igredients; i.e column 1 to 10
df3a=df3[["ingredients_1", "ingredients_2", "ingredients_3", "ingredients_4", "ingredients_5", "ingredients_6", "ingredients_7", "ingredients_8", "ingredients_9", "ingredients_10"]]
df3a

In [ ]:
df4=df3a.stack().reset_index(drop=True)
df5=df4.str.lower()

In [ ]:
df5.value_counts()
# we will group together English and French words

### Key takeaway (Question C)
* 5 most common ingredients are: sugar, water, citric acid & corn syrup

### D. Time Series
* Analyze the mean time difference between these two values
* Analyze the mean number of created items per month over the timeline of the data

In [ ]:
# Define a dataframe with key variables for this exercise: code, created_date, last_modified_date
tsdf= df2[['code','created_datetime','last_modified_datetime']]
tsdf.head()

In [ ]:
# create a new column to compute the time difference
tsdf['time_diff']=(tsdf['last_modified_datetime'] - tsdf['created_datetime'])

In [ ]:
tsdf.head()

In [ ]:
# check data types
tsdf.dtypes

In [ ]:
# Let's check the highest values
tsdf.sort_values('time_diff',ascending=False).head()

In [ ]:
tsdf['time_diff'].mean()
# the mean of the time difference is: 110 days 15:18:52.405461
# it idincates that there are probably some products which are up-dated on a regular basis since the beginning of the database

### Key takeaway
* Mean of the time difference is: 110 days 15:18:52

In [ ]:
# set the datetime as index to compute the mean of items created by time period
new=tsdf.set_index('created_datetime')
new.head()

In [ ]:
# define a fixed interval per month
by_month =new.groupby(new.index.month).count()
by_month

In [ ]:
# plot the data per month
by_month['code'].plot()

# labels
x=np.arange(12)+1
x_labels=['Jan.','Feb.','March','April','May','June','July','August','Sep.','Oct.','Nov.','Dec.']
plt.xticks(x, x_labels)

# set the title
plt.title('Average number of created products per month over the period 2012-2017', fontweight="bold")
plt.show()

### E. Build a database
* restrict your data to 1000 entries and 5 columns of your choice
* create a connection to a sqlite3 database
* create one or multiple tables, at least one of the tables should have a PRIMARY KEY
* fill the database with your data
* run at least one query to demonstrate that it works correctly


In [ ]:
# define a dataframe which will be used for filling the database
dbase=add_analysis_clean[['product_name', 'energy_100g','additives_n','proteins_100g','new_countries']][0:1000]
dbase.shape

In [ ]:
# create a database
db = sqlite3.connect('dbsubmission2.db')

# defining our helper function for running queries
def run_query(query):
    return pd.read_sql_query(query,db)

In [ ]:
# loading the data into the database
dbase.to_sql(name='dbsubmission2', con=db, if_exists='append', index=False)

In [ ]:
# checking that all data have been loaded
#checking that all the data was loaded
run_query("SELECT COUNT(*) FROM dbsubmission2")

In [ ]:
# check the first 5 rows of the table "dbsubmission2"
run_query("SELECT * FROM dbsubmission2 LIMIT 5;")

In [ ]:
# query: How many products with an energy level between 1200 and 2000?
query='''
SELECT energy_100g, COUNT(*) AS "Count" FROM dbsubmission2 
WHERE (energy_100g BETWEEN 1200 AND 2000)
limit 10;
'''
run_query(query) #314 products

In [ ]:
# What are the top 10 products with the highest level of energy and from which countries?
query='''
SELECT product_name, energy_100g, new_countries FROM dbsubmission2 
ORDER BY energy_100g DESC 
LIMIT 10;
'''
run_query(query)

### F. Finding correlations
* Identify the variables which most affect the nutritional score and provide some insight into which factors cause both a low or a high nutritional score

    #### Preliminary remarks
    * Nutrition score definition is available at: https://www.ndph.ox.ac.uk/cpnp/files/about/uk-ofcom-nutrient-profile-model.pdf
    * Points for foods and drinks fall on a scale from 1 to 100 where 1 is the least healthy and 100 is the most healthy


In [ ]:
# https://www.ndph.ox.ac.uk/cpnp/files/about/uk-ofcom-nutrient-profile-model.pdf

In [ ]:
# We take the datframe cleaned at the question A as a starting point
df2.head()

In [ ]:
# check number of products having a nutrition score 
df2['nutrition-score-uk_100g'].isnull().sum() # 53,199

In [ ]:
# drop NaN for nutrition-score-uk-100g
df2ns=df2.dropna(subset=['nutrition-score-uk_100g'])
df2ns.isnull().sum()

In [ ]:
df2ns

In [ ]:
df2ns.describe()
# Globally Mean and Standard deviation vary a lot from one variable to the other
# We would need to normalize data before performing the correlatin analysis

In [ ]:
# Let's have a look at the distribution plot
sns.distplot(df2ns['nutrition-score-uk_100g'])
plt.title('Displot nutrition score', fontweight="bold")
plt.rcParams["figure.figsize"]=[20,10]
# In order to focus the interpretation, we would consider 2 groups of products the "less healthy" having a NS from -10 to 10
# and " more healthy" products with a nutrition score from 10 to 30
# we may decide to discretize the NS if the normalization will not provide conclusive highlights

#### For the correlation matrix analysis we will consider the following criteria:
* A correlation matrix analysis performed on all numeric variables did not provide relevant results. We will adopt a more targeted approach based on the following principles:
    * Variables included in the calculation of the nutrition score (for which we have data): Energy, Saturated Fat, Total Sugar, Sodium, Fruit, Veg & Nuts (%), Fibre (g)
    * Other variables which we would assume being potentially relevant based on nutrition reports: number of additives...

#### Analysis of the overall nutrition score

In [ ]:
# select variables to perform the targeted correlation analysis
targ_corr=df2ns[['product_name',           
 'additives_n',
 'ingredients_from_palm_oil_n',
 'ingredients_that_may_be_from_palm_oil_n',
 'energy_100g',
 'energy-from-fat_100g',
 'fat_100g',
'saturated-fat_100g',
 'carbohydrates_100g',
 'sugars_100g',
'fiber_100g',
 'starch_100g',
 'proteins_100g',
 'salt_100g',
 'sodium_100g',
 'calcium_100g',
 'fruits-vegetables-nuts_100g',
 'fruits-vegetables-nuts-estimate_100g',
 'collagen-meat-protein-ratio_100g',
 'nutrition-score-uk_100g']]

In [ ]:
# change the index to get the product names
targ_corr.set_index('product_name', inplace=True)

In [ ]:
# replace NaN values by 0
targ_corr.fillna(0, inplace=True)

In [ ]:
# Compute the correlation matrix
corr = targ_corr.corr()
# source:https://datascience.stackexchange.com/questions/10459/calculation-and-visualization-of-correlation-matrix-with-pandas?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)
# define the style
corr.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '200px', 'font-size': '10pt'})\
    .set_precision(2)\

### Key takeways (overall nutrition score)
* Overall nutrition score is positively correlated to energy_100g (0.56), fat_100g (0.59), saturated fate (0.64) and sugars_100g to a certain extent (0.42)

#### Analysis of the low nutrition score (from -10 to +10)

In [ ]:
# We will analyse correlation matrix for low nutrition score; between -10 and +10
targ_corr_low= targ_corr[(targ_corr['nutrition-score-uk_100g']  >= 0) & (targ_corr['nutrition-score-uk_100g'] <= 10)]

In [ ]:
# replace NaN values by 0
targ_corr_low.fillna(0, inplace=True)

In [ ]:
# Compute the correlation matrix
corr = targ_corr_low.corr()
# source:https://datascience.stackexchange.com/questions/10459/calculation-and-visualization-of-correlation-matrix-with-pandas?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)
# define the style
corr.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_precision(2)\

### Key takeways (low nutrition score)
* From -10 to 10: correlation score are: energy 0.3, fat 0.37, sugar 0.36 - There are not very strong variables correlated to nutrition score
* From 0 to 10: energy 0.54, fat 0.36, carbohydrates 0.45, sugars 0.34 - Energy and carbohydrates present the highest correlation indicator. However, not very conclusive

#### Analysis of the high nutrition score (from 10 to 30)

In [ ]:
# We will analyse correlation matrix for low nutrition score; between -10 and +10
targ_corr_high= targ_corr[(targ_corr['nutrition-score-uk_100g']  > 10) & (targ_corr['nutrition-score-uk_100g'] <= 30)]

In [ ]:
# replace NaN values by 0
targ_corr_high.fillna(0, inplace=True)

In [ ]:
# Compute the correlation matrix
corr = targ_corr_high.corr()
# source:https://datascience.stackexchange.com/questions/10459/calculation-and-visualization-of-correlation-matrix-with-pandas?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)
# define the style
corr.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_precision(2)\

### Key takeways (high nutrition score)
* From >10 to 30: correlation score are: energy 0.27, fat 0.36, saturated fat 0.53 - saturated fat seems to present some correlation to nutrition score
* From 20 to 30: carbohydrates 0.24, sugars 0.24 - There are not very strong variables correlated to nutrition score

### End of the Project

### Back-up analysis

#### Correlation analysis with groups of food is not conclusive

In [ ]:
cat_corr=df2ns[['product_name','nutrition-score-uk_100g','pnns_groups_1','pnns_groups_2']]
cat_corr.head()

In [ ]:
# change the index to get the product names
cat_corr.set_index('product_name', inplace=True)

In [ ]:
# replace NaN values by 0
cat_corr.fillna(0, inplace=True)

In [ ]:
# create a dataframe (dummy) including dummy variable from the original "Region" variable
dummy1=pd.get_dummies(cat_corr['pnns_groups_1'])
dummy2=pd.get_dummies(cat_corr['pnns_groups_2'])

In [ ]:
# Concatanate cat_corr and dummy dataframes
cat_corrA=pd.concat([cat_corr, dummy1, dummy2], axis=1)
cat_corrA

In [ ]:
# drop pnns_groups_ 1 variable
cat_corr1=cat_corrA.drop('pnns_groups_1', axis=1)

In [ ]:
# drop pnns_groups_ 2 variable
cat_corr2=cat_corr1.drop('pnns_groups_2', axis=1)
cat_corr2

In [ ]:
cat_corr2.corr()

### Brute force correlation exercise

In [ ]:
# select variables to perform the correlation analysis
df2ns_corr=df2ns[['product_name',
 'additives_n',
 'ingredients_from_palm_oil_n',
 'ingredients_that_may_be_from_palm_oil_n',
 'energy_100g',
 'energy-from-fat_100g',
 'fat_100g',
 'saturated-fat_100g',
 '-caprylic-acid_100g',
 '-capric-acid_100g',
 '-lauric-acid_100g',
 '-myristic-acid_100g',
 '-palmitic-acid_100g',
 '-stearic-acid_100g',
 '-arachidic-acid_100g',
 '-behenic-acid_100g',
 '-montanic-acid_100g',
 'monounsaturated-fat_100g',
 'polyunsaturated-fat_100g',
 'omega-3-fat_100g',
 '-alpha-linolenic-acid_100g',
 '-eicosapentaenoic-acid_100g',
 '-docosahexaenoic-acid_100g',
 'omega-6-fat_100g',
 '-linoleic-acid_100g',
 '-arachidonic-acid_100g',
 '-gamma-linolenic-acid_100g',
 '-dihomo-gamma-linolenic-acid_100g',
 'omega-9-fat_100g',
 '-oleic-acid_100g',
 '-gondoic-acid_100g',
 'trans-fat_100g',
 'cholesterol_100g',
 'carbohydrates_100g',
 'sugars_100g',
 '-sucrose_100g',
 '-glucose_100g',
 '-fructose_100g',
 '-lactose_100g',
 '-maltose_100g',
 '-maltodextrins_100g',
 'starch_100g',
 'polyols_100g',
 'fiber_100g',
 'proteins_100g',
 'casein_100g',
 'serum-proteins_100g',
 'nucleotides_100g',
 'salt_100g',
 'sodium_100g',
 'alcohol_100g',
 'vitamin-a_100g',
 'beta-carotene_100g',
 'vitamin-d_100g',
 'vitamin-e_100g',
 'vitamin-k_100g',
 'vitamin-c_100g',
 'vitamin-b1_100g',
 'vitamin-b2_100g',
 'vitamin-pp_100g',
 'vitamin-b6_100g',
 'vitamin-b9_100g',
 'folates_100g',
 'vitamin-b12_100g',
 'biotin_100g',
 'pantothenic-acid_100g',
 'silica_100g',
 'bicarbonate_100g',
 'potassium_100g',
 'chloride_100g',
 'calcium_100g',
 'phosphorus_100g',
 'iron_100g',
 'magnesium_100g',
 'zinc_100g',
 'copper_100g',
 'manganese_100g',
 'fluoride_100g',
 'selenium_100g',
 'chromium_100g',
 'molybdenum_100g',
 'iodine_100g',
 'caffeine_100g',
 'taurine_100g',
 'ph_100g',
 'fruits-vegetables-nuts_100g',
 'fruits-vegetables-nuts-estimate_100g',
 'collagen-meat-protein-ratio_100g',
 'cocoa_100g',
 'carbon-footprint_100g',
 'nutrition-score-uk_100g']]

In [ ]:
# change the index to get the product names
df2ns_corr.set_index('product_name', inplace=True)

In [ ]:
# replace NaN values by 0
df2ns_corr.fillna(0, inplace=True)

In [ ]:
# Create normalizer
normalizer = Normalizer(norm='l2')

# Transform feature matrix
normalizer.transform(df2ns_corr)

In [ ]:
# Compute the correlation matrix
corr = df2ns_corr.corr()
# source:https://datascience.stackexchange.com/questions/10459/calculation-and-visualization-of-correlation-matrix-with-pandas?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)
# define the style
corr.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '200px', 'font-size': '10pt'})\
    .set_precision(2)\

In [ ]:
# energy 0.54, fat 0.59, saturated fat 0.64